In [2]:
def get_data(endpoint, params):
    URL = "https://v3.football.api-sports.io/"
    headers = {
	'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "fb2140228973d644db847895c454c22b"
    }
    response = requests.get(
        URL+endpoint,
        headers = headers,
        params = params
    )
    if response.status_code == 200:
            
        remaining = response.headers.get("x-ratelimit-requests-remaining")
        data = response.json()
        print(f"requests before reaching limit {remaining}")

    else:
        print(f"Error {response.status_code}, {response.text}")

    return data, remaining                                                                     

In [7]:
import requests

df, remain = get_data(endpoint, params)
data_sql = encode_data(df)

requests before reaching limit 99


In [8]:
df

{'get': 'teams',
 'parameters': {'league': '39', 'season': '2023'},
 'errors': {'requests': 'You have reached the request limit for the day, Go to https://dashboard.api-football.com to upgrade your plan.'},
 'results': 0,
 'paging': {'current': 1, 'total': 1},
 'response': []}

In [39]:
def encode_data(data_dict):
    encoded = []
    for key, val in data_dict.items():
        if isinstance(val, dict):
            encoded.extend(encode_data(val).items())
        elif isinstance(val, list):
            if val:
                if all(isinstance(i, dict) for i in val):
                    for k, v in enumerate(val):
                        encoded.extend(encode_data(v).items())
                else:
                    encoded.append((key, val))
            else:
                encoded.append((key, []))
        else:
            encoded.append((key, val))
    print(encoded)
    return dict(encoded)

In [45]:
import pandas as pd
listwa = []
final = []
for i in range(0,10):
    for j in range(0,10):
        d = (j, j+i)
        listwa.append(d)
        tmp = dict(listwa)
    final.append(tmp)
final


[{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9},
 {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10},
 {0: 2, 1: 3, 2: 4, 3: 5, 4: 6, 5: 7, 6: 8, 7: 9, 8: 10, 9: 11},
 {0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8, 6: 9, 7: 10, 8: 11, 9: 12},
 {0: 4, 1: 5, 2: 6, 3: 7, 4: 8, 5: 9, 6: 10, 7: 11, 8: 12, 9: 13},
 {0: 5, 1: 6, 2: 7, 3: 8, 4: 9, 5: 10, 6: 11, 7: 12, 8: 13, 9: 14},
 {0: 6, 1: 7, 2: 8, 3: 9, 4: 10, 5: 11, 6: 12, 7: 13, 8: 14, 9: 15},
 {0: 7, 1: 8, 2: 9, 3: 10, 4: 11, 5: 12, 6: 13, 7: 14, 8: 15, 9: 16},
 {0: 8, 1: 9, 2: 10, 3: 11, 4: 12, 5: 13, 6: 14, 7: 15, 8: 16, 9: 17},
 {0: 9, 1: 10, 2: 11, 3: 12, 4: 13, 5: 14, 6: 15, 7: 16, 8: 17, 9: 18}]

In [47]:
res = pd.DataFrame(final)
res

,0,1,2,3,4,5,6,7,8,9
0,0,1,2,3,4,5,6,7,8,9
1,1,2,3,4,5,6,7,8,9,10
2,2,3,4,5,6,7,8,9,10,11
3,3,4,5,6,7,8,9,10,11,12
4,4,5,6,7,8,9,10,11,12,13
5,5,6,7,8,9,10,11,12,13,14
6,6,7,8,9,10,11,12,13,14,15
7,7,8,9,10,11,12,13,14,15,16
8,8,9,10,11,12,13,14,15,16,17
9,9,10,11,12,13,14,15,16,17,18


In [13]:
import psycopg2

db_params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}

endpoint = 'teams'
params= {'league': 39,
         'season': 2023}


db_params = {
    'host': 'localhost',
    'database': 'preds',
    'user': 'postgres',
    'password': 'pass',
    'port': '5432'
}

data_to_sql('teams', data_sql, db_params)

Error: 'dict' object has no attribute 'columns'


In [11]:
def data_to_sql(table_name, df, db_params):
    conn = None
    cur = None
    try:
        # Establish the connection
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()
        
        #insert data into tables
        modified_columns = [change_column_name(col) for col in df.columns]
        insert_query = """
            INSERT INTO {} ({})
            VALUES ({})
        """.format(table_name, ','.join(df.columns), ','.join(['%s']*len(df.columns)))
        pgcursor.executemany(insert_query, df.values.tolist())
        print(f'table {table_name} updated')
        
        # Commit the changes
        conn.commit()
        
    
    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        if conn is not None:
            # Close the cursor and connection
            cur.close()
        if cur is not None:
            conn.close()